In [1]:
import torch
import Unetwork
import os
import diffusers
import cv2


f:\Anaconda\envs\nc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
modelConfig = {
'device': torch.device("cpu"),
'batch_size': 8,
'T': 100,
'channel': 16,
'channel_mult': [2,2,2,2],
'atten': [2,3],
'num_res_block': 3,                  #不知道是多少，随便写一个吧
'dropout': 0.2,                      #dropout置零率，在0到1之间
'training_load_weight': None,        #填从那个文件夹里加载权重
'eval_load_weight': '40_.pt',  #填从那个文件夹里加载权重
'save_weight_dir': 'E:/D1/diffusion/my_models/diff_test/weight/',#把权重保存在那个文件夹里
'picture_save_path': 'E:/D1/diffusion/my_models/diff_test/picture/',#把生成的图片保存在那个文件夹里
'lr': 0.0001,
'epoch': 100,                        #不知道是多少，x修改了这个注释
'multiplier': 1.0,                   #不知道填啥
'bata_1':0.0001,
'bata_T':0.02,
'grad_clip': 2.0,
}

In [3]:



net_model = Unetwork.UNet(
    T=modelConfig["T"],
    in_ch=13,
    ch=modelConfig["channel"],
    ch_mult=modelConfig["channel_mult"],
    atten=modelConfig["atten"],
    num_res_block=modelConfig["num_res_block"],
    dropout=modelConfig["dropout"]
)

ckpt = torch.load(os.path.join(
            modelConfig["save_weight_dir"],
            modelConfig["eval_load_weight"]), map_location=modelConfig["device"])

net_model.load_state_dict(ckpt)
net_model.to(modelConfig["device"])
net_model.eval()
conditions = torch.zeros((1, 10, 128, 128)).to(modelConfig["device"])


In [6]:
xt = torch.randn((1, 3, 128, 128)).to(modelConfig["device"])
ddim_sch = diffusers.schedulers.scheduling_ddim.DDIMScheduler(num_train_timesteps=100, beta_schedule='squaredcos_cap_v2', clip_sample=False)
ddim_sch.set_timesteps(20)

for t in ddim_sch.timesteps:
    t = t.to(modelConfig["device"]).unsqueeze(0)
    print(t)
    eps = net_model(xt, t, conditions)
    xt = ddim_sch.step(eps, t, xt).prev_sample

    cv2.imwrite(modelConfig['picture_save_path']+str(t.item())+'test.png', ((xt[0].cpu().detach().numpy().transpose(1, 2, 0) + 1) * 127.5).astype('uint8'))


tensor([95])
tensor([90])
tensor([85])
tensor([80])
tensor([75])
tensor([70])
tensor([65])
tensor([60])
tensor([55])
tensor([50])
tensor([45])
tensor([40])
tensor([35])
tensor([30])
tensor([25])
tensor([20])
tensor([15])
tensor([10])
tensor([5])
tensor([0])
